In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

from scipy.stats import chi2_contingency, spearmanr, skew, pearsonr
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ParameterGrid

#Mô hình XG Boost
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import RandomizedSearchCV

#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/Healthcare insurance fraud/Changed Datasets/Inner/Inpatient-Fraud-Dataset.csv')
data

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,Yes
1,BENE17521,CLM34721,2009-01-20,2009-02-01,PRV55912,19000,PHY349293,PHY370861,PHY363291,2009-01-20,45340,1068.0,2009-02-01,987,4240,2639,2948,40390,45821,28489,5854,2753,E9305,NaN,7769.0,5849.0,NaN,NaN,NaN,NaN,1913-12-01,NaN,2,1,0,39,230,12,12,1,1,1,2,1,2,2,1,2,2,2,19000,1068,100,20,Yes
2,BENE21718,CLM72336,2009-10-17,2009-11-04,PRV55912,17000,PHY334706,PHY334706,NaN,2009-10-17,V5789,1068.0,2009-11-04,941,V5789,4168,73313,7812,7993,78830,72273,43812,4019,NaN,9338.0,NaN,NaN,NaN,NaN,NaN,1922-10-01,NaN,1,1,0,39,600,12,12,2,2,2,2,2,2,2,1,1,2,2,17000,1068,1050,540,Yes
3,BENE22934,CLM73394,2009-10-25,2009-10-29,PRV55912,13000,PHY390614,PHY323689,PHY363291,2009-10-25,71946,1068.0,2009-10-29,506,71535,71960,4019,V1202,4240,2449,2768,NaN,NaN,NaN,8154.0,NaN,NaN,NaN,NaN,NaN,1930-07-01,NaN,2,1,0,39,280,12,12,2,1,2,2,1,2,1,1,1,1,2,27000,2136,450,160,Yes
4,BENE24402,CLM32911,2009-01-08,2009-01-12,PRV55912,3000,PHY380413,PHY432598,NaN,2009-01-08,1748,1068.0,2009-01-12,593,2330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8543.0,NaN,NaN,NaN,NaN,NaN,1925-09-01,NaN,2,2,0,39,510,12,12,1,2,2,1,2,2,2,1,2,2,2,3000,1068,8900,740,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40469,BENE156181,CLM57017,2009-06-24,2009-07-01,PRV55706,12000,PHY319644,PHY319644,PHY403799,2009-06-24,78097,1068.0,2009-07-01,940,78079,45341,V1251,41401,2449,2930,78830,1985,0414,NaN,9955.0,NaN,NaN,NaN,NaN,NaN,1952-03-01,NaN,1,1,Y,37,260,12,12,1,1,1,1,2,2,1,1,2,1,1,20000,2136,740,320,No
40470,BENE156872,CLM67045,2009-09-06,2009-09-13,PRV51572,5000,PHY407588,PHY311768,PHY311768,2009-09-06,486,1068.0,2009-09-13,190,486,7905,V5867,5990,2859,7904,49121,41401,2639,NaN,3326.0,NaN,NaN,NaN,NaN,NaN,1943-03-01,NaN,2,5,Y,5,200,12,12,2,2,1,2,1,1,1,1,2,2,1,5000,1068,2670,360,No
40471,BENE157414,CLM76036,2009-11-15,2009-11-23,PRV55688,14000,PHY409302,NaN,NaN,2009-11-15,4280,1068.0,2009-11-23,308,42821,E9342,42731,5856,4280,5859,51881,V0481,2761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1939-12-01,NaN,2,1,Y,37,450,12,12,1,1,1,2,1,1,1,1,2,1,2,26790,2136,70,0,No
40472,BENE158281,CLM35143,2009-01-23,2009-01-23,PRV56485,10000,PHY367360,PHY367360,NaN,2009-01-23,7823,1068.0,2009-01-23,238,41071,78552,5856,V4581,5789,39891,40390,2866,NaN,NaN,9390.0,NaN,NaN,NaN,NaN,NaN,1957-11-01,NaN,1,1,Y,44,430,12,12,2,1,1,1,1,2,1,1,2,2,2,10300,1068,5390,1730,No


# **GIẢI THÍCH DỮ LIỆU**

**BeneID:** Mã định danh (ID) của người thụ hưởng bảo hiểm (bệnh nhân).

**ClaimID:** Mã định danh của yêu cầu bảo hiểm, tương ứng với một sự kiện y tế hoặc điều trị cụ thể mà bệnh nhân đã yêu cầu bồi thường.

**ClaimStartDt:** Ngày bắt đầu của yêu cầu bảo hiểm (ngày bắt đầu điều trị).

**ClaimEndDt:** Ngày kết thúc của yêu cầu bảo hiểm (ngày kết thúc điều trị).

**Provider:** Mã định danh của cơ sở y tế hoặc nhà cung cấp dịch vụ y tế.

**InscClaimAmtReimbursed:** Số tiền mà bảo hiểm đã bồi hoàn cho yêu cầu bảo hiểm đó.

**AttendingPhysician:** Mã định danh của bác sĩ điều trị chính (bác sĩ trực tiếp chịu trách nhiệm điều trị cho bệnh nhân).

**OperatingPhysician:** Mã định danh của bác sĩ phẫu thuật (nếu có phẫu thuật liên quan).

**OtherPhysician:** Mã định danh của các bác sĩ khác liên quan đến quá trình điều trị.

**AdmissionDt:** Ngày nhập viện của bệnh nhân (đối với bệnh nhân nội trú).

**ClmAdmitDiagnosisCode:** Mã chẩn đoán bệnh ban đầu khi bệnh nhân nhập viện.

**DeductibleAmtPaid:** Số tiền khấu trừ mà bệnh nhân phải tự trả trước khi bảo hiểm bắt đầu chi trả.

**DischargeDt:** Ngày xuất viện của bệnh nhân (đối với bệnh nhân nội trú).

**DiagnosisGroupCode:** Mã nhóm chẩn đoán, mô tả nhóm bệnh lý hoặc điều kiện sức khỏe của bệnh nhân.

**ClmDiagnosisCode_1** đến **ClmDiagnosisCode_10:** Các mã chẩn đoán khác nhau được liên kết với yêu cầu bảo hiểm này. Mỗi mã đại diện cho một chẩn đoán hoặc điều kiện y tế mà bệnh nhân đã được xác định trong quá trình điều trị.

**ClmProcedureCode_1** đến **ClmProcedureCode_6:** Các mã thủ tục y tế được thực hiện cho bệnh nhân trong quá trình điều trị.

**DOB:** Ngày sinh của bệnh nhân.

**DOD:** Ngày mất của bệnh nhân (nếu có).

**Gender:** Giới tính của bệnh nhân (thường là 1: Nam, 2: Nữ).

**Race:** Chủng tộc của bệnh nhân (có thể được mã hóa dưới dạng số, ví dụ: 1 là Người da trắng, 2 là Người Mỹ gốc Phi, v.v.).

**RenalDiseaseIndicator:** Chỉ số có bệnh thận (thường là 1: Có, 0: Không).

**State:** Mã tiểu bang nơi bệnh nhân cư trú.

**County:** Mã quận nơi bệnh nhân cư trú.

**NoOfMonths_PartACov:** Số tháng bệnh nhân đã có bảo hiểm phần A (bảo hiểm bệnh viện) trong suốt năm tài chính.

**NoOfMonths_PartBCov:** Số tháng bệnh nhân đã có bảo hiểm phần B (bảo hiểm y tế) trong suốt năm tài chính.

**ChronicCond_Alzheimer:** Có bị bệnh Alzheimer hay không (1: Có, 0: Không).

**ChronicCond_Heartfailure:** Có bị suy tim hay không.

**ChronicCond_KidneyDisease:** Có bị bệnh thận mãn tính hay không.

**ChronicCond_Cancer:** Có bị ung thư hay không.

**ChronicCond_ObstrPulmonary:** Có bị bệnh phổi tắc nghẽn mãn tính hay không.

**ChronicCond_Depression:** Có bị trầm cảm hay không.

**ChronicCond_Diabetes:** Có bị bệnh tiểu đường hay không.

**ChronicCond_IschemicHeart:** Có bị bệnh tim thiếu máu cục bộ hay không.

**ChronicCond_Osteoporasis:** Có bị loãng xương hay không.

**ChronicCond_rheumatoidarthritis:** Có bị viêm khớp dạng thấp hay không.

**ChronicCond_stroke:** Có bị đột quỵ hay không.

**IPAnnualReimbursementAmt:** Tổng số tiền bảo hiểm đã hoàn trả cho các yêu cầu nội trú trong năm.

**IPAnnualDeductibleAmt:** Tổng số tiền khấu trừ bệnh nhân phải trả cho các yêu cầu nội trú trong năm.

**OPAnnualReimbursementAmt:** Tổng số tiền bảo hiểm đã hoàn trả cho các yêu cầu ngoại trú trong năm.

**OPAnnualDeductibleAmt:** Tổng số tiền khấu trừ bệnh nhân phải trả cho các yêu cầu ngoại trú trong năm.

**PotentialFraud:** Cột này đánh dấu liệu yêu cầu bảo hiểm có tiềm ẩn dấu hiệu gian lận hay không (có thể là 1: Có, 0: Không).

# **FILLING NULL COLUMNS**

***Trước hết, cần thêm vào một số columns và xóa bớt một số columns sau khi đã thêm columns mới, bao gồm:***

In [4]:
#Chuyển đổi 2 columns sau thành dạng datetime, thêm một cột mới để tính "Số ngày điều trị":
def adding_new_columns(data):
    data['ClaimStartDt'] = pd.to_datetime(data['ClaimStartDt'], format='%Y-%m-%d', errors='coerce')
    data['ClaimEndDt'] = pd.to_datetime(data['ClaimEndDt'], format='%Y-%m-%d', errors='coerce')
    data['DOB'] = pd.to_datetime(data['DOB'], format='%Y-%m-%d', errors='coerce')
    data['DOD'] = pd.to_datetime(data['DOD'], format='%Y-%m-%d', errors='coerce')
    data['AdmissionDt'] = pd.to_datetime(data['AdmissionDt'], format='%Y-%m-%d', errors='coerce')
    data['DischargeDt'] = pd.to_datetime(data['DischargeDt'], format='%Y-%m-%d', errors='coerce')

    # Tạo cột mới 'Valuable_claim_time(days)', 'years_ols' và 'dead_or_alive' tính số ngày điều trị, số tuổi của bệnh nhân, tình trạng sống chết
    data['Valuable_claim_time(days)'] = (data['ClaimEndDt'] - data['ClaimStartDt']).dt.days
    data['years_old'] = (data['DischargeDt'] - data['DOB']).dt.days // 365
    data['dead_or_alive'] = data['DOD'].apply(lambda x: 0 if pd.isna(x) else 1)
    data['treatment_time(days)'] = (data['DischargeDt'] - data['AdmissionDt']).dt.days

    # Loại bỏ các cột không cần thiết
    data = data.drop(columns=['ClaimStartDt', 'ClaimEndDt', 'DOB', 'DOD', 'DischargeDt', 'AdmissionDt'])

    return data

data = adding_new_columns(data)

In [5]:
#Xóa bớt những columns có nhiều Null:
def dropping_null_cols(data):
  dropping_null_cols = []
  for i in data.columns:
    if data[i].isnull().sum() > int(0.5*(data.shape[0])):
      dropping_null_cols.append(i)
  data = data.drop(columns = dropping_null_cols)
  return data

data = dropping_null_cols(data)

In [6]:
#Các column sau sẽ fillna bằng '0', có nghĩa là bệnh nhân không sử dụng dịch vụ này (thay vì dropna do thiếu):
def fillna_some_cols(data):
  for i in ['AttendingPhysician', 'OperatingPhysician', 'DeductibleAmtPaid', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4',
            'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmProcedureCode_1']:
            if i == 'DeductibleAmtPaid':
              data[i] = data[i].fillna(0)
            else:
              data[i] = data[i].fillna('None')
  return data
data = fillna_some_cols(data)

# **ENCODING DATA**

In [7]:
category_cols = ['Provider', 'AttendingPhysician', 'OperatingPhysician', 'ClmAdmitDiagnosisCode', 'DiagnosisGroupCode', 'ClmDiagnosisCode_1',
                 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
                 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmProcedureCode_1', 'Gender', 'Race', 'RenalDiseaseIndicator',
                 'State', 'County', 'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
                 'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
                 'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke', 'dead_or_alive']

numeric_cols = ['InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov', 'IPAnnualReimbursementAmt',
                'IPAnnualDeductibleAmt', 'OPAnnualDeductibleAmt', 'Valuable_claim_time(days)', 'years_old', 'treatment_time(days)']

***Kiểm định Chi-squared cho các columns phân loại***

In [8]:
remove_obj_cols =[]

#Chi-square test cho các cột Object so với cột Output của data
for i in category_cols:
  chi2, pval, _, _ = chi2_contingency(pd.crosstab(data[i], data['PotentialFraud']))
  print(i, '---', pval)
  if pval > 0.05:
    remove_obj_cols.append(i)
  else:
    continue
print(remove_obj_cols)

Provider --- 0.0
AttendingPhysician --- 0.0
OperatingPhysician --- 0.0
ClmAdmitDiagnosisCode --- 0.28444352849783405
DiagnosisGroupCode --- 0.8046232696698308
ClmDiagnosisCode_1 --- 0.3289967319870451
ClmDiagnosisCode_2 --- 0.23747852403212694
ClmDiagnosisCode_3 --- 0.20898171792778597
ClmDiagnosisCode_4 --- 0.12152179773758424
ClmDiagnosisCode_4 --- 0.12152179773758424
ClmDiagnosisCode_5 --- 0.3588888445520672
ClmDiagnosisCode_6 --- 0.6619101360460519
ClmDiagnosisCode_7 --- 0.9879268230268861
ClmDiagnosisCode_8 --- 0.4447973325072163
ClmDiagnosisCode_9 --- 0.20118071779008306
ClmProcedureCode_1 --- 0.6235525622466414
Gender --- 0.059825578313804244
Race --- 0.7105631970892268
RenalDiseaseIndicator --- 0.6408446258234877
State --- 0.0
County --- 0.0
ChronicCond_Alzheimer --- 0.42857121678374877
ChronicCond_Heartfailure --- 0.08197730826606105
ChronicCond_KidneyDisease --- 0.6824093640446098
ChronicCond_Cancer --- 0.6337751731500292
ChronicCond_ObstrPulmonary --- 0.44993094886741825
Chr

In [9]:
data = data.drop(columns = remove_obj_cols)
category_cols = [i for i in category_cols if i not in remove_obj_cols]

***Tương tự, Spearman Correlation cho các biến liên tục***

In [10]:
spear_cols = numeric_cols
spear_cols.append('PotentialFraud')
spearman_df = data[spear_cols]
correlation_results = []

# Duyệt qua tất cả các cột trong DataFrame
for col in spearman_df.columns:
  if col == 'PotentialFraud':
    continue
  else:
    corr, p_value = spearmanr(spearman_df[col], spearman_df['PotentialFraud'])
    correlation_results.append((col, corr, p_value))

# Chuyển kết quả thành DataFrame để dễ quan sát
correlation_df = pd.DataFrame(correlation_results, columns=['Feature', 'Spearman Correlation', 'p-value'])
correlation_df = correlation_df.sort_values(by='Spearman Correlation', ascending=False)
correlation_df

,Feature,Spearman Correlation,p-value
0,InscClaimAmtReimbursed,0.020219,0.000047
4,IPAnnualReimbursementAmt,0.019210,0.000111
6,OPAnnualDeductibleAmt,0.008245,0.097154
5,IPAnnualDeductibleAmt,0.004929,0.321348
1,DeductibleAmtPaid,0.004006,0.420294
2,NoOfMonths_PartACov,0.002662,0.592288
3,NoOfMonths_PartBCov,0.001923,0.698834
8,years_old,0.001559,0.753772
7,Valuable_claim_time(days),-0.001944,0.695793
9,treatment_time(days),-0.002074,0.676445


In [11]:
spearman_cols_to_delete = ['OPAnnualDeductibleAmt', 'IPAnnualDeductibleAmt', 'DeductibleAmtPaid', 'NoOfMonths_PartACov',
                            'NoOfMonths_PartBCov', 'years_old', 'Valuable_claim_time(days)', 'treatment_time(days)']
data = data.drop(columns = spearman_cols_to_delete)
numeric_cols = [i for i in numeric_cols if i not in spearman_cols_to_delete]

In [12]:
data = data.drop(columns = ['BeneID', 'ClaimID', 'AttendingPhysician', 'OperatingPhysician', 'Provider'])

***Encoding***

In [13]:
data['PotentialFraud'] = data['PotentialFraud'].map({'Yes': 1, 'No': 0})

In [14]:
data

,InscClaimAmtReimbursed,State,County,IPAnnualReimbursementAmt,OPAnnualReimbursementAmt,PotentialFraud
0,26000,39,230,36000,60,1
1,19000,39,230,19000,100,1
2,17000,39,600,17000,1050,1
3,13000,39,280,27000,450,1
4,3000,39,510,3000,8900,1
...,...,...,...,...,...,...
40469,12000,37,260,20000,740,0
40470,5000,5,200,5000,2670,0
40471,14000,37,450,26790,70,0
40472,10000,44,430,10300,5390,0


# **PREDICTION**

In [15]:
#Hàm đánh giá hiệu suất
def print_scores(y_test, y_pred):
  print(classification_report(y_test, y_pred))

#Hàm chia tập train-test và MinMax Scaler
def preprocess_data(data, test_size=0.2, random_state=42):

    # Chia tập train-test
    X = data.drop(columns=['PotentialFraud'], axis=1)
    features = X.columns

    y = data['PotentialFraud']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    #Standard Scaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

In [16]:
X_train, X_test, y_train, y_test = preprocess_data(data)

***Random Forest***

*Overfitting*

In [17]:
model_RF = RandomForestClassifier(n_estimators=200, random_state=42, class_weight = 'balanced')
model_RF.fit(X_train, y_train)

y_train_pred = model_RF.predict(X_train)
y_test_pred = model_RF.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     13599
           1       1.00      0.99      0.99     18780

    accuracy                           0.99     32379
   macro avg       0.99      0.99      0.99     32379
weighted avg       0.99      0.99      0.99     32379

              precision    recall  f1-score   support

           0       0.62      0.53      0.57      3473
           1       0.68      0.76      0.72      4622

    accuracy                           0.66      8095
   macro avg       0.65      0.64      0.64      8095
weighted avg       0.66      0.66      0.65      8095



***2.XGBOOST***

*Ổn nhưng chưa cao lắm*

In [18]:
model_XGB = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB.fit(X_train, y_train)

y_train_pred = model_XGB.predict(X_train)
y_test_pred = model_XGB.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.84      0.72      0.77     13599
           1       0.82      0.90      0.85     18780

    accuracy                           0.82     32379
   macro avg       0.83      0.81      0.81     32379
weighted avg       0.82      0.82      0.82     32379

              precision    recall  f1-score   support

           0       0.65      0.55      0.60      3473
           1       0.70      0.78      0.74      4622

    accuracy                           0.68      8095
   macro avg       0.68      0.66      0.67      8095
weighted avg       0.68      0.68      0.68      8095



***3.GridSearch***

In [19]:
param_grid = {
  'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
  'learning_rate'   : [0.001, 0.01, 0.1, 1],
  'max_depth'       : [3, 5, 8, 10],
  'alpha'           : [1, 10, 100],
  'n_estimators'    : [10, 50, 100]
}

index = 0

answers_grid = {
    'combination'       :[],
    'train_Accuracy'    :[],
    'test_Accuracy'     :[],
    'colsample_bytree'  :[],
    'learning_rate'     :[],
    'max_depth'         :[],
    'alpha'             :[],
    'n_estimators'      :[]
}

In [23]:
# Tạo một lưới các tham số
param_list = list(ParameterGrid(param_grid))

for params in param_list:
    # Tạo mô hình với các tham số hiện tại
    model_XGB_new = xgb.XGBClassifier(
        objective='binary:logistic',  # Sử dụng mục tiêu phân loại nhị phân
        colsample_bytree=params['colsample_bytree'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        alpha=params['alpha'],
        n_estimators=params['n_estimators']
    )

    # Huấn luyện mô hình
    model_XGB_new.fit(X_train, y_train)

    # Dự đoán trên tập huấn luyện và kiểm tra
    y_train_pred = model_XGB_new.predict(X_train)
    y_test_pred = model_XGB_new.predict(X_test)

    # Tính toán độ chính xác
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Lưu trữ kết quả
    answers_grid['combination'].append(params)
    answers_grid['train_Accuracy'].append(train_accuracy)
    answers_grid['test_Accuracy'].append(test_accuracy)
    answers_grid['colsample_bytree'].append(params['colsample_bytree'])
    answers_grid['learning_rate'].append(params['learning_rate'])
    answers_grid['max_depth'].append(params['max_depth'])
    answers_grid['alpha'].append(params['alpha'])
    answers_grid['n_estimators'].append(params['n_estimators'])

# In kết quả tham số tốt nhất dựa trên độ chính xác tập kiểm tra
best_index = answers_grid['test_Accuracy'].index(max(answers_grid['test_Accuracy']))
print("Best Hyperparameters:", answers_grid['combination'][best_index])

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

Best Hyperparameters: {'alpha': 1, 'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.58      0.22      0.32     13599
           1       0.61      0.88      0.72     18780

    accuracy                           0.61     32379
   macro avg       0.59      0.55      0.52     32379
weighted avg       0.60      0.61      0.55     32379

              precision    recall  f1-score   support

           0       0.58      0.22      0.32      3473
           1       0.60      0.88      0.71      4622

    accuracy                           0.60      8095
   macro avg       0.59      0.55      0.52      8095
weighted avg       0.59      0.60      0.54      8095



# **Kiểm tra tập test**

In [24]:
data_cols = data.columns.to_list()
test_cols = data_cols.copy()
test_cols.remove('PotentialFraud')

In [25]:
test = pd.read_csv('/content/drive/MyDrive/Datasets/Healthcare insurance fraud/Changed Datasets/Inner/Inpatient-Fraud-Test-Dataset.csv')
test = test[test_cols]
test

,InscClaimAmtReimbursed,State,County,IPAnnualReimbursementAmt,OPAnnualReimbursementAmt
0,9000,45,780,21260,120
1,3000,45,910,8000,2540
2,7000,45,780,10200,40
3,5000,45,910,5000,600
4,9000,45,910,9000,10
...,...,...,...,...,...
9546,7000,32,50,7080,0
9547,8000,33,20,18500,0
9548,7000,10,650,13000,100
9549,6000,10,650,13000,100


In [26]:
#Dự đoán tập test:
y_pred_test = model_XGB.predict(test)

In [29]:
y_pred_test = pd.Series(y_pred_test)
y_pred_test.value_counts()

,count
1,8176
0,1375
